In [1]:
%load_ext autoreload
%autoreload 2
from new_aeb_gplvm import *
from utils.data_generator import DataGenerator
from utils.myutils import Utils
from tqdm import trange
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import matplotlib.pyplot as plt
import random
from tqdm import tqdm_notebook
random.seed(42)
import json

2023-11-27 20:18:05.263429: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-27 20:18:05.263460: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-27 20:18:05.264532: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-27 20:18:05.271110: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-27 20:18:06.100159: W tensorflow/comp

In [2]:
datagenerator = DataGenerator()
utils = Utils()

In [3]:
# dataset and model list / dict

In [4]:
def create_dataframe(dataset):
    datagenerator.dataset = dataset
    data = datagenerator.generator(
        la=0.999,
        realistic_synthetic_mode=None,
        stdscale=True,
        minmax=False,
        noise_type=None,
    )
    Y_train = torch.tensor(data["X_train"], dtype=torch.float32)
    Y_test = torch.tensor(data["X_test"], dtype=torch.float32)
    lb_train = torch.tensor(data["y_train"], dtype=torch.float32)
    lb_test = torch.tensor(data["y_test"], dtype=torch.float32)
    idx_n = np.where(lb_train == 0)[0]
    Y_train = Y_train[idx_n]
    lb_train = lb_train[idx_n]
    return Y_train, Y_test, lb_train, lb_test

In [ ]:
with open("experiment_results_normal_gplvm.json") as file:
    experiments = json.load(file)
for experiment in tqdm_notebook(experiments):
    if "auc_roc" not in experiment.keys():
        Y_train, Y_test, lb_train, lb_test = create_dataframe(experiment["dataset"])
        n_train = len(Y_train)
        data_dim = Y_train.shape[1]
        latent_dim = experiment["latent_dim"]
        nn_layers = tuple(map(int, experiment["layers"].split(",")))
        kernel = experiment["kernel"]
        
        n_inducing = 50
        n_epochs = 1000
        lr = 0.01
        batch_size = 128
        
        try:
            gplvm = AD_GPLVM(latent_dim, n_inducing, n_epochs, nn_layers, lr, batch_size)
            gplvm.fit(Y_train)
            score = gplvm.predict_score(Y_test)
            metrics = utils.metric(y_true=lb_test, y_score=score)
            
            experiment["auc_roc"] = metrics['aucroc']
            experiment["auc_pr"] = metrics['aucpr']
        except:
            experiment['auc_roc'] = 0.00
            experiment['auc_pr'] = 0.00
        
        experiment["lr"] = lr
        experiment["batch_size"] = batch_size
        experiment["n_inducing"] = n_inducing
        experiment["n_epochs"] =  n_epochs
        
        with open("experiment_results_normal_gplvm.json", "w") as file:
            json.dump(experiments, file)

/tmp/ipykernel_14166/372986365.py:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for experiment in tqdm_notebook(experiments):


  0%|          | 0/188 [00:00<?, ?it/s]

subsampling for dataset 03_backdoor...
current noise type: None
{'Samples': 10000, 'Features': 196, 'Anomalies': 236, 'Anomalies Ratio(%)': 2.36}
Kernel não escolhido!



Loss: 198.2, iter no: 500: 100%|██████████| 1000/1000 [01:12<00:00, 13.97it/s]
                                                                              /home/ec2-user/SageMaker/OE-GPLVM/new_aeb_gplvm.py:302: UserWarning: The use of `x.T` on tensors of dimension other than 2 to reverse their shape is deprecated and it will throw an error in a future release. Consider `x.mT` to transpose batches of matrices or `x.permute(*torch.arange(x.ndim - 1, -1, -1))` to reverse the dimensions of a tensor. (Triggered internally at /opt/conda/conda-bld/pytorch_1686274778240/work/aten/src/ATen/native/TensorShape.cpp:3571.)
  klu_expanded = ll_shape.T.add_(klu).sum(-1).T.div((self.n_train))


subsampling for dataset 03_backdoor...
current noise type: None
{'Samples': 10000, 'Features': 196, 'Anomalies': 236, 'Anomalies Ratio(%)': 2.36}
Kernel não escolhido!


...


Loss: 23.59, iter no: 500: 100%|██████████| 1000/1000 [00:16<00:00, 59.21it/s]
                                                                              

current noise type: None
{'Samples': 2114, 'Features': 21, 'Anomalies': 466, 'Anomalies Ratio(%)': 22.04}
Kernel não escolhido!



Loss: 23.68, iter no: 500: 100%|██████████| 1000/1000 [00:16<00:00, 59.19it/s]
                                                                              

current noise type: None
{'Samples': 2114, 'Features': 21, 'Anomalies': 466, 'Anomalies Ratio(%)': 22.04}
Kernel não escolhido!



Loss: 23.99, iter no: 500: 100%|██████████| 1000/1000 [00:17<00:00, 57.53it/s]
                                                                              